In [1]:
!python --version

Python 3.12.4


# LangChain: Models, Prompts and Output Parsers

## Outline

 * Direct API calls to Gemini
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [2]:
# !pip install -r requirements.txt

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os

api_key = os.getenv("GOOGLE_API_KEY")

### Chat Model

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(
    google_api_key=api_key,
    model="gemini-2.0-flash-exp",
    temperature=1
)

C:\Users\91889\anaconda3\envs\langchainenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prompt template

In [6]:
template_string = """\
Translate the text that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```\
"""

`Note:` \ is used to avoid new line (\n) in the template

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [8]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [9]:
prompt_template.messages[0].prompt.template

'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```'

In [10]:
customer_style = """American English \
in a calm and respectful tone\
"""

In [11]:
customer_email = """\
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!\
"""

In [12]:
customer_message = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)

In [13]:
customer_message[0].content

"Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```"

In [14]:
type(customer_message[0])

langchain_core.messages.human.HumanMessage

In [15]:
response = chat_model.invoke(customer_message)

In [16]:
response.content

"Okay, I understand you're upset that the lid of your blender came off and made a mess on your kitchen walls with smoothie. That's definitely frustrating! And it's even more disappointing that the warranty won't cover the cost of cleaning it up. I'll do my best to help you out. What kind of assistance are you looking for specifically?\n"

In [17]:
# Another example
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!\
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [18]:
service_message = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)

In [19]:
response = chat_model.invoke(service_message)

response.content

"Ahoy there, matey!\n\nAvast, it seems ye be havin' a bit o' a pickle with yer kitchen, aye? Now, regarding the warranty, I be havin' to inform ye that it unfortunately don't cover the costs o' cleanin' up that mess. Seems the blame for that there blender mishap rests squarely on yer shoulders, what with forgettin' the lid afore settin' sail, so to speak. A bit o' a tough break, I must admit. Fair winds to ye on yer next endeavor!\n"

### Output Parsers

**Example**
```bash
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [20]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [21]:
prompt_template = ChatPromptTemplate.from_template(review_template)

message = prompt_template.format_messages(text=customer_review)

message[0].content

"For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n\n"

In [22]:
response = chat_model.invoke(message)

print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}



In [23]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a JSON format

In [24]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [25]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [26]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [27]:
format_instructions = output_parser.get_format_instructions()

In [28]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [29]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [30]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

message = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [31]:
# print(message[0].content)

In [32]:
response = chat_model.invoke(message)

In [33]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "['It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.']"
}
```



In [34]:
output_dict = output_parser.parse(response.content)

In [35]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "['It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.']"}

In [36]:
type(output_dict)

dict

In [37]:
output_dict.get('delivery_days')

'2'

In [46]:
# Using JsonOutputParser

In [47]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()

In [61]:
response = chat_model.invoke(message)

result = output_parser.parse(response.content)

result

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "['It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.']"}

In [56]:
type(result)

dict

In [58]:
result.get('gift')

'True'

In [59]:
result.get('delivery_days')

'2'

In [60]:
result.get('price_value')

"['It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.']"